In [1]:
from llm_client import invoke_llm
from features.iternary_generation.llm_parser import llm_parse_user_input

In [2]:
prompt = "Plan a 5-day foodie + nightlife trip to Goa with 40k budget"

In [13]:
from features.iternary_generation.llm_parser import llm_parse_user_input
from features.iternary_generation.iternary_generator import generate_itinerary
from features.iternary_generation.basic_tag_personalization import apply_personalization
from features.maps_scrapper.scraper import fetch_google_reviews
from features.maps_scrapper.scraper import preprocess_google_reviews


In [4]:
prompt = "Plan a 5-day foodie + nightlife trip to Goa with 40k budget"

In [5]:
parsed = llm_parse_user_input(prompt)
print("Parsed Input:", parsed)

place = parsed["location"]

Prompt: 

        Extract the following fields from the trip request:
        - Location (city or region in India)
        - Duration in days
        - Budget in INR
        - Themes (heritage, nightlife, adventure, food, nature, general)

        Input: "Plan a 5-day foodie + nightlife trip to Goa with 40k budget"
        
        Return ONLY valid JSON (no markdown, no explanation).
        {
        "location": "...",
        "duration_days": <int>,
        "budget": <int>,
        "themes": [ ... ]
        }
    
Client Created successfully
```json
{
"location": "Goa",
"duration_days": 5,
"budget": 40000,
"themes": ["food", "nightlife"]
}
```
Parsed Input: {'location': 'Goa', 'duration_days': 5, 'budget': 40000, 'themes': ['food', 'nightlife']}


In [6]:
import sys
print(sys.executable)
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en_core_web_sm


d:\tripPlanner\trip-planner-genai\backend\.venv\Scripts\python.exe



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB 93.1 kB/s eta 0:02:18
     --------------------------------------- 0.0/12.8 MB 108.9 kB/s eta 0:01:58
     --------------------------------------- 0.0/12.8 MB 108.9 kB/s eta 0:01:58
     --------------------------------------- 0.0/12.8 MB 108.9 kB/s eta 0:01:58
     ---------------------------------------- 0.0/12.8 MB 98.1 kB/s eta 0:02:11
     --------------------------------------- 0.1/12.8 MB 130.9 kB/s eta 0:01:38
     --------------------------------------- 0.1/12.8 MB 130.9 kB/s eta 0:01:38
     --------------------------------------- 0.1/12.8 MB 174.7 kB/s eta 0:01:13
      -------------------------------------- 0.2/12.8 MB 317.5 kB/s eta 


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import spacy
nlp = spacy.load("en_core_web_sm")


In [8]:
def extract_spots_from_itinerary(itinerary):
    spots = set()
    for day in itinerary:
        for key in ["morning", "afternoon", "evening"]:
            if key in day:
                doc = nlp(day[key])
                for ent in doc.ents:
                    if ent.label_ in ["GPE", "LOC", "FAC"]:  # city, location, facility
                        spots.add(ent.text)
    return list(spots)

In [9]:
initial_summary = "Popular attractions and tips for visitors."
initial_itinerary = generate_itinerary(parsed, initial_summary)

Client Created successfully
```json
{
  "itinerary": [
    {
      "day": 1,
      "morning": "Arrive at Goa International Airport (GOI). Take a pre-booked taxi/cab to your accommodation in Anjuna. Check into your hotel. Grab a relaxed breakfast at a local shack on Anjuna Beach, trying out Goan Poha or Chicken Xacuti with pao.",
      "afternoon": "Explore the Anjuna Flea Market (if it's Wednesday, otherwise explore the local shops and boutiques). Lunch at a popular local eatery like 'Thalassa' for delicious Greek food with a view, or 'German Bakery' for a more relaxed vibe and baked goods.",
      "evening": "Start your evening with sunset drinks at Curlies Beach Shack, known for its vibrant atmosphere and beautiful views. Later, head to 'Hilltop' (if there's an event) or 'Shiva Valley' for classic psytrance nightlife, a quintessential Goan experience. Dinner at Curlies or a nearby beach shack for fresh seafood.",
      "estimated_cost": 7500
    },
    {
      "day": 2,
      "mornin

In [10]:
spots = extract_spots_from_itinerary(initial_itinerary["itinerary"])

In [11]:
print("Extracted Spots:", spots)

Extracted Spots: ['Curlies Beach Shack', 'Anjuna', 'Mambo', 'Arambol Beach', 'Assagao', 'Goa International Airport', 'Arambol beach', 'Arambol', 'Banyan', 'Arpora', 'Fontainhas', 'Mandovi River', 'Anjuna Beach', 'Goa', 'Vagator Beach']


In [18]:
# 4️⃣ Fetch Google reviews for each spot
all_reviews_text = []
for spot in spots:
    reviews = fetch_google_reviews(spot, limit=5)
    all_reviews_text.append(reviews)

DEBUG: Searching for place: Curlies Beach Shack
DEBUG SERPAPI_KEY exists: True
DEBUG: Full response keys: dict_keys(['search_metadata', 'search_parameters', 'search_information', 'place_results'])
DEBUG: Full response: {
  "search_metadata": {
    "id": "68cc1ff0970121208f0c02a5",
    "status": "Success",
    "json_endpoint": "https://serpapi.com/searches/1ea66f734cde2507/68cc1ff0970121208f0c02a5.json",
    "created_at": "2025-09-18 15:06:24 UTC",
    "processed_at": "2025-09-18 15:06:24 UTC",
    "google_maps_url": "https://www.google.com/maps/search/Curlies+Beach+Shack?hl=en",
    "raw_html_file": "https://serpapi.com/searches/1ea66f734cde2507/68cc1ff0970121208f0c02a5.html",
    "total_time_taken": 0.9
  },
  "search_parameters": {
    "engine": "google_maps",
    "type": "search",
    "q": "Curlies Beach Shack",
    "google_domain": "google.com",
    "hl": "en"
  },
  "search_information": {
    "local_results_state": "Showing results for type: \"place\" instead of type: \"search\""

In [ ]:
for review in all_reviews:
    print(review.get("text", review.get("snippet", "")))
    print("-" * 50)  # optional separator


[[{'link': 'https://www.google.com/maps/reviews/data=!4m8!14m7!1m6!2m5!1sCi9DQUlRQUNvZENodHljRjlvT2t3ME1WaDVaVzFzYTNsek1rNHhhVU5LUTNGTkxXYxAB!2m1!1s0x0:0x1f949df8c252d491!3m1!1s2@1:CAIQACodChtycF9oOkw0MVh5ZW1sa3lzMk4xaUNKQ3FNLWc%7C%7C?hl=en-US', 'rating': 4.0, 'date': '2 weeks ago', 'iso_date': '2025-09-02T18:07:48Z', 'iso_date_of_last_edit': '2025-09-02T18:07:48Z', 'images': ['https://lh3.googleusercontent.com/geougc-cs/AB3l90CgD78nHSne5tP1g4HzciSkCdU5b5Im4NqAiUqcJQ68jdwrIZK0YBLhd-Il1wKxjhDAn3_Er5PwId_6qQl7OZabV6GPEth0iweOoS71LOyLWYC8uFNgzcAcvgCqpn_Yy81y5O26WJ5rFwI', 'https://lh3.googleusercontent.com/geougc-cs/AB3l90Ai5s9NlD78lHcp-Jia_pwoNOh8na4nbctL8RV51IkagX-xcRhxMzVkWbFz23wqgJuaxk6Hu2O6c90nneYyAHTDGu1-IVKO5J1geqokHVVk7CFbrqjp2z6Fw0qsbCg7YhOjzue8OpfA0JGr', 'https://lh3.googleusercontent.com/geougc-cs/AB3l90Afbgvy6XrGI9RqCOqbYfJLdLANFV6nL8ZRq8uAaXbVgx10xgwTcKu7R-T7A4O46F4IzFOY4MGrveuWUKftzZaIkA16xO5fMt2XBNExVb3nRV9fi4SYC8rqY8DZ3bGopMj89DMz8Sk9WTY'], 'source': 'Google', 'review_id': 